<a href="https://colab.research.google.com/github/halim-jun/Ai_study/blob/master/202008_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
import pandas as pd
train=pd.read_csv("train.csv")

In [ ]:
test=pd.read_csv("test.csv")
test_2=pd.read_csv("gender_submission.csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


#1. Data 전처리

##1) 테스트 데이터 전처리
- cabin, Name, ticket number, passenger number drop
- X, Y set 으로 분리

In [ ]:
total_test=pd.merge(test, test_2, how='inner', left_on='PassengerId', right_on='PassengerId')
total_test.dropna(axis=0, how='any', inplace=True)
total_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
12,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S,1
14,906,1,"Chaffee, Mrs. Herbert Fuller (Carrie Constance...",female,47.0,1,0,W.E.P. 5734,61.1750,E31,S,1
24,916,1,"Ryerson, Mrs. Arthur Larned (Emily Maria Borie)",female,48.0,1,3,PC 17608,262.3750,B57 B59 B63 B66,C,1
26,918,1,"Ostby, Miss. Helene Ragnhild",female,22.0,0,1,113509,61.9792,B36,C,1
28,920,1,"Brady, Mr. John Bertram",male,41.0,0,0,113054,30.5000,A21,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
404,1296,1,"Frauenthal, Mr. Isaac Gerald",male,43.0,1,0,17765,27.7208,D40,C,0
405,1297,2,"Nourney, Mr. Alfred (Baron von Drachstedt"")""",male,20.0,0,0,SC/PARIS 2166,13.8625,D38,C,0
407,1299,1,"Widener, Mr. George Dunton",male,50.0,1,1,113503,211.5000,C80,C,0
411,1303,1,"Minahan, Mrs. William Edward (Lillian E Thorpe)",female,37.0,1,0,19928,90.0000,C78,Q,1


In [ ]:
y_test=total_test['Survived']
y_test

x_test=total_test.drop(labels=['Cabin','Name','Ticket', 'PassengerId'], inplace=False, axis=1)


In [ ]:
x_test.drop(labels=['Survived'], inplace=True, axis=1)

In [ ]:
x_test.replace(to_replace='S', value=0, inplace=True)
x_test.replace(to_replace='C', value=1, inplace=True)
x_test.replace(to_replace='Q', value=2, inplace=True)
x_test.replace(to_replace='male', value=1, inplace=True)
x_test.replace(to_replace='female', value=0, inplace=True)

In [ ]:
x_test.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
Survived    0
dtype: int64

##2. Train data 전처리 
1) 기본 
- null, na 제거
- 불필요한 칼럼 제거

In [ ]:
import sklearn

In [ ]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
train.drop(labels='Cabin', axis=1, inplace=True)

In [ ]:
train_withage=train.dropna(axis=0, how='any', inplace=False)
train_withage.isna().sum()
train.drop()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

## 2) Accuracy Paradox 확인

In [ ]:
train_withage['Survived'].describe()
train_withage.replace(to_replace='male', value=1, inplace=True)
train_withage.replace(to_replace='female', value=0, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [ ]:
train_withage['Embarked'].unique()
train_withage.replace(to_replace='S', value=0, inplace=True)
train_withage.replace(to_replace='C', value=1, inplace=True)
train_withage.replace(to_replace='Q', value=2, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


## 3) Label, Feature 분리

In [ ]:
from sklearn.model_selection import train_test_split
label=train_withage['Survived']
feature=train_withage.drop(labels='Survived', axis=1, inplace=False)

In [ ]:
feature.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

(178, 7)

#2. Model selection
- Best accuracy 인 classifier model 로 선정

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model_tree=DecisionTreeClassifier(random_state=32)
model_tree.fit(feature, label)
y_pred=model_tree.predict(x_test)
tree_acc=accuracy_score(y_pred, y_test)

In [ ]:
tree_acc

0.8275862068965517

In [ ]:
model_svm=SVC()
model_svm.fit(feature, label)
y_pred=model_svm.predict(x_test)
svm_acc=accuracy_score(y_test, y_pred)

In [ ]:
model_rand=RandomForestClassifier()
model_rand.fit(feature, label)
y_pred=model_rand.predict(x_test)
rand_acc=accuracy_score(y_test, y_pred)

In [ ]:
model_Logi=LogisticRegression()
model_Logi.fit(feature, label)
y_pred=model_Logi.predict(x_test)
Logi_acc=accuracy_score(y_test, y_pred)

In [ ]:
model_SGD=SGDClassifier()
model_SGD.fit(feature, label)
y_pred=model_SGD.predict(x_test)
SGD_acc=accuracy_score(y_test, y_pred)

In [ ]:
pd.DataFrame([['tree',tree_acc], ['svm',svm_acc], ['random',rand_acc], ['Logistic regressopn',Logi_acc], ['Stochastic Gradient Logisitc',SGD_acc]], columns=['Algorithm', 'Accuracy'])

,Algorithm,Accuracy
0,tree,0.827586
1,svm,0.620690
2,random,0.885057
3,Logistic regressopn,0.850575
4,Stochastic Gradient Logisitc,0.724138
